In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture --no-stderr
!pip install python-dotenv langchain_openai langchain-chroma pypdf langchain langchain_community

In [ ]:
# 환경변수 설정

In [ ]:
# 라이브러리 불러오기
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")

# **2024 부동산 보고서 RAG 챗봇**

In [ ]:
# 라이브러리 불러오기
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_opneai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

# **PDF 문서 로드 및 텍스트 분할**

In [ ]:
loader = PyPDFLoader("pdf 경로")
# PDF 파일에서 문서 로드
documents = loader.load()
# 텍스트 분할 설정: 청크 크기와 겹칩 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# 문서를 작은 청크로 분할
chunks = text_splitter.split_documents(documents)
# 분할된 청크 수 확인
print('분할된 청크의 수:', len(chunks))

# **임베딩 생성 및 Chroma 데이터베이스 저장**

In [ ]:
# 임베딩 생성 및 Chroma 데이터베이스에 저장
# OpenAI의 임베딩 모델 설정
embedding_function = OpenAIEmbeddings()
persist_directory = "크로마 DB 저장 경로"

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_function,
    # DB 저장 경로
    persist_directory=persist_directory,
)

print('문서의 수:', vectorstore._collection.count())

# **검색 및 재정렬**

In [ ]:
# 리트리버 생성
# 상위 3개의 관련 문서 검색 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
# 프롬프트 템플릿 설정: 사용자 질문에 대한 답변을 생성하기 위한 템플릿
template = """당신은 KB 부동산 보고서 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요.
컨텍스트: {context}
"""

# 프롬프트 템플릿 설정
prompt = ChatPromptTempalte.from_messages(
    [
        ("system", template),
        ("placeholder", "{chat_history}"),
        ("human", "{question}")
    ]
)

# AI 모델 설정
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [ ]:
print(prompt.format(context="컨텍스트 예시"), chat_history=["대화 기록 예시1", "대화 기록 예시2"], question="질문 예시")

# **문서 형식 변환 함수 정의**

In [ ]:
def format_docs(docs):
  # 문서를 줄바꿈으로 연결
  return "\n\n".join(doc.page_content for doc in docs)

# 체인 구성: 검색한 문서를 프롬프트에 연결하고 모델을 통해 응답 생성
chain = (
    # 검색된 문서를 연결하여 전달
    RunnablePassthrough.assign(
        context=lambda x: format_docs(retriever.invoke(x["question"]))
    )
    | prompt
    | model
    # 결과를 문자열로 변환
    | StrOutputParser()
)

# **대화 기록을 유지하기 위한 메모리 설정**

In [ ]:
chat_history = ChatMessageHistory()

chain_with_memory = RunnableWithMessageHistory(
    chain,
    # 세션 ID별 대화기록 생성
    lambda session_id: chat_history,
    input_messages_key="question",
    history_messages_key="chat_history"
)

# **챗봇 실행 함수 정의**

In [ ]:
def chat_with_bot():
  session_id = "user_session"
  print("KB 부동산 보고서 챗봇입니다. 질문해 주세요. (종료하려면 'quit' 입력)")

  while True:
    user_input = input("사용자: ")

    if user_input.lower() == 'quit':
      break

    # 사용자의 질문에 따라 chain_with_memory를 통해 응답 생성
    response = chain_with_memory.invoke(
        {"question": user_input},
        {"configurable": {"session_id": session_id}}
    )

    print("챗봇:", response)

# **스트림릿 적용**

In [ ]:
# 라이브러리 설치
%%capture --no-stderr
!pip install streamlit pyngrok

In [ ]:
# ngrok 인증키 설정
!ngrok config add-authtoken 인증키

# **app.py**

In [1]:
import os
import streamlit as st
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

# 환경 변수 로드
load_dotenv("/content/.env")
api_key = os.getenv("OPENAI_API_KEY")

# PDF 처리 함수
@st.cache_resource
def process_pdf():
  loader = PyPDFLoader("PDF 경로")
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

  return text_splitter.split_documents(documents)

# 벡터 저장소 초기화
@st.cache_resource
def initialize_vectorstore():
  chunks = process_pdf()
  embeddings = OpenAIEmbeddings(openai_api_key=api_key)

  return Chroma.from_documents(chunks, embeddings)

# 체인 초기화
@st.cache_resource
def initialize_chain():
  vertorstore = initialize_vectorstore()
  retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

  template = """당신은 KB 부동산 보고서 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요.

  컨텍스트: {context}
  """

  prompt = ChatPromptTemplate.from_messages([
      ("system", template),
      ("placeholder", "{chat_history}"),
      ("human", "{question}")
  ])

  model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, openai_api_key=api_key)

  def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

  base_chain = (
      RunnablePassthrough.assign(
          context=lambda x: format_docs(retriever.invoke(x["question"]))
      )
      | prompt
      | model
      | StrOutputParser(0)
  )

  return RunnableWithMessageHistory(
      base_chain,
      lambda session_id: ChatMessageHistory()
      input_messages_key="question",
      history_messages_key="chat_history"
  )

# Streamlit UI
def main():
  st.set_page_config(page_title="KB 부동산 보고서 챗봇", page_icon="🏠")
  st.title("🏠 KB 부동산 보고서 AI 어드바이저")
  st.caption("2024 KB 부동산 보고서 기반 질의응답 시스템")

  # 세션 상태 초기화
  if "messages" not in st.session_state:
    st.session_state.messages = []

  # 채팅 기록 표시
  for message in st.session_state.messages:
    with st.chat_message(message["role"]):
      st.markdown(message["content"])

  # 사용자 입력 처리
  if prompt := st.chst_input("부동산 관련 질문을 입력하세요."):
    # 사용자 메시지 표시
    with st.chat_message("user"):
      st.markdown(prompt)
    st.session_state.messages.append({"role": "user", "content": prompt})

    # 체인 초기화
    chain = initialize_chain()

    # AI 응답 생성
    with st.chat_message("assistant"):
      with st.spinner("답변 생성 중..."):
        response = chain.invoke(
            {"question": prompt},
            {"configurable": {"session_id": "streamlit_session"}}
        )

        st.markdown(response)

    st.session_state.messages.append({"role": "assistant", "content": response})

if __name__ == "__main__":
  main()


SyntaxError: invalid syntax. Perhaps you forgot a comma? (ipython-input-50788963.py, line 67)